In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import pickle

IMG_SIZE = 64
DATASET_DIR = "/content/drive/MyDrive/leapGestRecog/"

def load_data():
    X, y = [], []
    label_map = {}
    label_id = 0

    for subject in os.listdir(DATASET_DIR):
        subject_path = os.path.join(DATASET_DIR, subject)
        if not os.path.isdir(subject_path): continue

        for gesture_folder in os.listdir(subject_path):
            gesture_path = os.path.join(subject_path, gesture_folder)
            if not os.path.isdir(gesture_path): continue

            if gesture_folder not in label_map:
                label_map[gesture_folder] = label_id
                label_id += 1

            for img_file in os.listdir(gesture_path):
                img_path = os.path.join(gesture_path, img_file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
                X.append(img)
                y.append(label_map[gesture_folder])

    return np.array(X), np.array(y), label_map

X, y, label_map = load_data()
X = X.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(y.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test))

model.save("gesture_model.h5")
with open("label_map.pkl", "wb") as f:
    pickle.dump(label_map, f)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
501/501 ━━━━━━━━━━━━━━━━━━━━ 158s 302ms/step - accuracy: 0.5212 - loss: 1.6143 - val_accuracy: 0.5452 - val_loss: 1.3565
Epoch 2/15
501/501 ━━━━━━━━━━━━━━━━━━━━ 203s 305ms/step - accuracy: 0.9435 - loss: 0.1616 - val_accuracy: 0.9968 - val_loss: 0.0110
Epoch 3/15
501/501 ━━━━━━━━━━━━━━━━━━━━ 195s 292ms/step - accuracy: 0.9669 - loss: 0.0912 - val_accuracy: 0.9890 - val_loss: 0.0405
Epoch 4/15
501/501 ━━━━━━━━━━━━━━━━━━━━ 203s 293ms/step - accuracy: 0.9695 - loss: 0.0846 - val_accuracy: 0.9978 - val_loss: 0.0101
Epoch 5/15
501/501 ━━━━━━━━━━━━━━━━━━━━ 144s 288ms/step - accuracy: 0.9794 - loss: 0.0625 - val_accuracy: 0.9983 - val_loss: 0.0050
Epoch 6/15
501/501 ━━━━━━━━━━━━━━━━━━━━ 202s 289ms/step - accuracy: 0.9850 - loss: 0.0439 - val_accuracy: 0.9968 - val_loss: 0.0105
Epoch 7/15
501/501 ━━━━━━━━━━━━━━━━━━━━ 143s 285ms/step - accuracy: 0.9855 - loss: 0.0477 - val_accuracy: 0.9958 - val_loss: 0.0334
Epoch 8/15
501/501 ━━━━━━━━━━━━━━━━━━━━ 207s 296ms/step - accuracy: 0.9870 -